# 3 Preprocessing<a id='3_Preprocessing'></a>

## 3.1 Contents<a id='3.1_Contents'></a>
* [3 Preprocessing](#3_Preprocessing)
  * [3.1 Contents](#3.1_Contents)
  * [3.2 Introduction](#3.2_Introduction)
    * [3.2.1 Recap of Data Science Problem](#3.2.1_Recap_of_Data_Science_Problem)
    * [3.2.2 Introduction To Notebook](#3.2.2_Introduction_To_Notebook)
  * [3.3 Objectives](#3.3_Objectives)
  * [3.4 Imports](#3.4_Imports)
  * [3.5 Load Data](#3.5_Load_Data)
  * [3.6 Filling in missing values with moving average](#3.6_Filling_in_missing_values_with_moving_average)
  * [3.7 Merge building data with processed weather data](#3.7_Merge_building_data_with_processed_weather_data)
  * [3.8 Add additional features](#3.8_Add_additional_features)
  * [3.9 Set aside validation data](#3.9_Set_aside_validation_data)
  * [3.10 Split data into training and testing data](#3.10_Split_data_into_training_and_testing_data)
    * [3.10.1 Subsample the data for linear regression and random forest models](#3.10.1_Subsample_the_data_for_linear_regression_and_random_forest_models)

## 3.2 Introduction<a id='3.2_Introduction'></a>

### 3.2.1 Recap of Data Science Problem<a id='3.2.1_Recap_of_Data_Science_Problem'></a>

The goal of this project is to predict the building energy consumption baseline

### 3.2.2 Introduction To Notebook<a id='3.2.2_Introduction_To_Notebook'></a>

The datasets are the following:
* meter readings with energy use in kWh per hour for the year 2016. This dataset contains the building ID, the meter type (e.g., chilled water, electricity, steam, and hot water), and the meter reading.
* Details about each building. This includes the primary use of the building (e.g., education, office, etc.), number of floors, square footage, year it was built, and a site ID that links to weather data
* Weather data. This includes precipitation, cloud coverage, air temperature, dew temperature, sea level pressure, and wind direction and speed.

## 3.3 Objectives<a id='3.3_Objectives'></a>

* Fill in missing values in weather data with the moving average of the weather features over time
* Add additional features such as day of the week, month, hour, and season, and convert these to cyclic features
* Set aside validation data to test the final model with
* Split the data into training and testing data
* Obtain subsamples from the training and testing sets to use for the linear and random forest regression models

## 3.4 Imports<a id='3.4_Imports'></a>

In [1]:
#Import pandas, matplotlib.pyplot, and seaborn in the correct lines below
import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import dates
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import datetime
import math
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

## 3.5 Load Data<a id='3.5_Load_Data'></a>

In [2]:
# the supplied CSV data files are in the data/processed directory
#load data
weather_data=pd.read_csv('../data/processed/weather_data_clean.csv')
building_and_meter_data=pd.read_csv('../data/processed/building_and_meter_data.csv')

# drop unnecessary columns
building_and_meter_data=building_and_meter_data.drop('Unnamed: 0',axis=1)
weather_data=weather_data.drop('Unnamed: 0',axis=1)

#add kWh/square_feet column
building_and_meter_data['consumption_per_square_feet']=building_and_meter_data['meter_reading']/building_and_meter_data['square_feet']

# change the format of timestamp to datetime
building_and_meter_data['timestamp']=pd.to_datetime(building_and_meter_data['timestamp'],format='%Y-%m-%d %H:%M:%S')
weather_data['timestamp']=pd.to_datetime(weather_data['timestamp'],format='%Y-%m-%d %H:%M:%S')

select electricity meter readings only

In [3]:
by_meter=(building_and_meter_data['meter']==0)
electricity_data=building_and_meter_data[by_meter]

## 3.6 Filling in missing values with moving average<a id='3.6_Filling_in_missing_values_with_moving_average'></a>

In [4]:
site_ids=weather_data.site_id.unique()
weather_features=weather_data.columns[2:9]
for site_id in site_ids:
    by_site_id=(weather_data['site_id']==site_id)
    weather_by_site_id=weather_data[by_site_id]
    for feature in weather_features:
        weather_data.loc[weather_data['site_id']==site_id,feature]=weather_by_site_id[feature].interpolate('index').rolling(168).mean()

Drop unnecessary colums

In [5]:
weather_data=weather_data.drop(['precip_depth_1_hr_log','cloud_coverage_log','wind_speed_log'],axis=1)
electricity_data=electricity_data.drop(['meter_reading_log','floor_count_log','square_feet_log'],axis=1)

## 3.7 Merge building data with processed weather data<a id='3.7_Merge_building_data_with_processed_weather_data'></a>

In [6]:
data=electricity_data.merge(weather_data, on=['timestamp','site_id'])

## 3.8 Add additional features<a id='3.8_Add_additional_features'></a>

Here, I am going to do some more feature engineering. I am adding season as a feature, and cyclic features for the month, day, and hour features

In [15]:
seasons = {3:(datetime.datetime(2016,6,21), datetime.datetime(2016,9,22)),
           4:(datetime.datetime(2016,9,23), datetime.datetime(2016,12,20)),
           2:(datetime.datetime(2016,3,21), datetime.datetime(2016,6,20))}

def get_season(date):
    for season,(season_start, season_end) in seasons.items():
        if date>=season_start and date<= season_end:
            return season
    else:
        return 1

In [16]:
data['season']=data['timestamp'].apply(get_season)

In [17]:
data["cos_hour"] = np.cos(2 * math.pi * data["hour"] / data["hour"].max())
data["sin_hour"] = np.sin(2 * math.pi * data["hour"] / data["hour"].max())
data["cos_month"] = np.cos(2 * math.pi * data["month"] / data["month"].max())
data["sin_month"] = np.sin(2 * math.pi * data["month"] / data["month"].max())
data["cos_day"] = np.cos(2 * math.pi * data["day"] / data["day"].max())
data["sin_day"] = np.sin(2 * math.pi * data["day"] / data["day"].max())
data["cos_dayofweek"] = np.cos(2 * math.pi * data["dayofweek"] / data["dayofweek"].max())
data["sin_dayofweek"] = np.sin(2 * math.pi * data["dayofweek"] / data["dayofweek"].max())
data["cos_season"] = np.cos(2 * math.pi * data["season"] / data["season"].max())
data["sin_season"] = np.sin(2 * math.pi * data["season"] / data["season"].max())

## 3.9 Set aside validation data<a id='3.9_Set_aside_validation_data'></a>

Here I want to set aside meter readings from buildings 1298 and 283 to validate the final chosen model

In [18]:
#bring aside building 1298 for validation
validation_data=data[data['building_id']==1298]
validation_data.to_csv('../data/processed/electricity/validation_data.csv')
data=data[data['building_id']!=1298]

In [28]:
#bring aside building 1298 for validation
validation_data=data[data['building_id']==283]
validation_data.to_csv('../data/processed/electricity/validation_data_283.csv')
#data=data[data['building_id']!=1298]

## 3.10 Split data into training and testing data<a id='3.10_Split_data_into_training_and_testing_data'></a>

Because I have a large dataset, here I am taking subsamples of 10,000 and 100,000 rows from the large dataset to use these subsamples for training the linear and random forest regression models. But before doing so, I am splitting the data into training and testing data, then taking the subsamples from the training data. 

In [20]:
X=data.drop(columns='meter_reading')
y=data.meter_reading
# In the first step we will split the data in training and remaining dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7, random_state=47)

### 3.10.1 Subsample the data for linear regression and random forest models<a id='3.10.1_Subsample_the_data_for_linear_regression_and_random_forest_models'></a>

In [21]:
X_train_subsample_lr=X_train.sample(100000,random_state=123)
X_test_subsample_lr=X_test.sample(100000,random_state=123)
y_train_subsample_lr=y_train.sample(100000,random_state=123)
y_test_subsample_lr=y_test.sample(100000,random_state=123)

X_train_subsample_rf=X_train.sample(10000,random_state=123)
X_test_subsample_rf=X_test.sample(10000,random_state=123)
y_train_subsample_rf=y_train.sample(10000,random_state=123)
y_test_subsample_rf=y_test.sample(10000,random_state=123)

In [22]:
X_train_subsample_lr.to_csv('../data/processed/electricity/subsamples/linear/X_train_subsample_lr.csv')
X_test_subsample_lr.to_csv('../data/processed/electricity/subsamples/linear/X_test_subsample_lr.csv')
y_train_subsample_lr.to_csv('../data/processed/electricity/subsamples/linear/y_train_subsample_lr.csv')
y_test_subsample_lr.to_csv('../data/processed/electricity/subsamples/linear/y_test_subsample_lr.csv')

X_train_subsample_rf.to_csv('../data/processed/electricity/subsamples/random_forest/X_train_subsample_rf.csv')
X_test_subsample_rf.to_csv('../data/processed/electricity/subsamples/random_forest/X_test_subsample_rf.csv')
y_train_subsample_rf.to_csv('../data/processed/electricity/subsamples/random_forest/y_train_subsample_rf.csv')
y_test_subsample_rf.to_csv('../data/processed/electricity/subsamples/random_forest/y_test_subsample_rf.csv')

In [23]:
X_train.to_csv('../data/processed/electricity/X_train.csv')
X_test.to_csv('../data/processed/electricity/X_test.csv')
y_train.to_csv('../data/processed/electricity/y_train.csv')
y_test.to_csv('../data/processed/electricity/y_test.csv')

In [24]:
data.to_csv('../data/processed/electricity/elec_data.csv')